In [233]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
from fuzzywuzzy import fuzz
import missingno as msno

In [234]:
read = pd.read_csv('..\data\processed\\rentals_cleaned.csv')

In [235]:
rentals = read.copy()

In [236]:
rentals.shape

(266220, 30)

In [237]:
rentals.head()

,state,serviceCharge,heatingType,telekomTvOffer,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,yearConstructed,...,typeOfFlat,zip_code,noRooms,thermalChar,numberOfFloors,garden,district,town_municipality,date,baseRent_per_sqm
0,Nordrhein_Westfalen,245.00,central_heating,ONE_YEAR_FREE,False,False,6,4.62,10.0,1965-01-01,...,ground_floor,44269,4.0,181.4,3.0,True,Dortmund,Schüren,2019-05-01,6.918605
1,Rheinland_Pfalz,134.00,self_contained_central_heating,ONE_YEAR_FREE,False,True,8,3.47,10.0,1871-01-01,...,ground_floor,67459,3.0,NaN,NaN,False,Rhein_Pfalz,Böhl_Iggelheim,2019-05-01,8.988764
2,Sachsen,255.00,floor_heating,ONE_YEAR_FREE,True,True,8,2.72,2.4,2019-01-01,...,apartment,1097,3.0,NaN,4.0,False,Dresden,Äußere_Neustadt_Antonstadt,2019-10-01,11.515513
3,Sachsen,58.15,district_heating,ONE_YEAR_FREE,False,True,9,1.53,40.0,1964-01-01,...,other,9599,3.0,86.0,NaN,False,Mittelsachsen,Freiberg,2019-05-01,5.898538
4,Bremen,138.00,self_contained_central_heating,NaN,False,True,19,2.46,NaN,1950-01-01,...,apartment,28213,3.0,188.9,NaN,False,Bremen,Neu_Schwachhausen,2020-02-01,9.003178


In [238]:
read2 = pd.read_csv('..\data\processed\\state_features.csv')

In [239]:
state = read2.copy()

In [240]:
state

,state,area_km2,population_2019,population_per_km2,gdp_per_capita_2018,hdi
0,Baden-Württemberg,35752,11100394,310,47290,0.953
1,Bavaria(Bayern),70552,13124737,185,48323,0.947
2,Berlin,892,3669491,4086,41967,0.950
3,Brandenburg,29479,2521893,85,29541,0.914
4,Bremen,419,681202,1630,49215,0.951
5,Hamburg,755,1847253,2439,66879,0.975
6,Hesse(Hessen),21115,6288080,297,46923,0.949
7,Lower Saxony(Niedersachsen),47609,7993448,168,38423,0.922
8,Mecklenburg-Vorpommern,23180,1609675,69,28940,0.910
9,North Rhine-Westphalia(Nordrhein-Westfalen),34085,17932651,526,39678,0.936


In [241]:
state_supply = rentals.groupby(['state']).agg(total_state_listings = pd.NamedAgg(column = 'baseRent', aggfunc = 'size'), \
                                               total_state_sqm = pd.NamedAgg(column = 'livingSpace', aggfunc = 'sum'))

In [242]:
state_supply

,total_state_listings,total_state_sqm
state,,
Baden_Württemberg,15961,1342806.70
Bayern,21490,1718920.03
Berlin,10271,842022.42
Brandenburg,6875,474904.52
Bremen,2940,213666.12
Hamburg,3730,312253.87
Hessen,17682,1466268.24
Mecklenburg_Vorpommern,6580,416557.76
Niedersachsen,16404,1261969.68


In [243]:
state_supply.reset_index(inplace = True)
state_supply

,state,total_state_listings,total_state_sqm
0,Baden_Württemberg,15961,1342806.70
1,Bayern,21490,1718920.03
2,Berlin,10271,842022.42
3,Brandenburg,6875,474904.52
4,Bremen,2940,213666.12
5,Hamburg,3730,312253.87
6,Hessen,17682,1466268.24
7,Mecklenburg_Vorpommern,6580,416557.76
8,Niedersachsen,16404,1261969.68
9,Nordrhein_Westfalen,62069,4615660.71


In [244]:
#View the names of states from our geo_economy dataframe
stateNames_geo_economy = state.state
stateNames_geo_economy

0                               Baden-Württemberg
1                                 Bavaria(Bayern)
2                                          Berlin
3                                     Brandenburg
4                                          Bremen
5                                         Hamburg
6                                   Hesse(Hessen)
7                     Lower Saxony(Niedersachsen)
8                          Mecklenburg-Vorpommern
9     North Rhine-Westphalia(Nordrhein-Westfalen)
10          Rhineland-Palatinate(Rheinland-Pfalz)
11                                       Saarland
12                                Saxony(Sachsen)
13                  Saxony-Anhalt(Sachsen-Anhalt)
14                             Schleswig-Holstein
15                           Thuringia(Thüringen)
Name: state, dtype: object

In [245]:
#View the names of states from our rentals_eda dataframe
stateNames_rentals_eda = list(set(rentals.state))
stateNames_rentals_eda

['Bayern',
 'Nordrhein_Westfalen',
 'Thüringen',
 'Berlin',
 'Hessen',
 'Saarland',
 'Baden_Württemberg',
 'Hamburg',
 'Rheinland_Pfalz',
 'Schleswig_Holstein',
 'Sachsen_Anhalt',
 'Sachsen',
 'Mecklenburg_Vorpommern',
 'Niedersachsen',
 'Brandenburg',
 'Bremen']

In [246]:
#Change state names from geo_economy dataframe to state names from rentals_eda
for geo_name in stateNames_geo_economy:
    for rentals_name in stateNames_rentals_eda:
        if fuzz.WRatio(geo_name, rentals_name) > 80:
            state.state = state.state.replace(geo_name, rentals_name)
            
state.state  

0          Baden_Württemberg
1                     Bayern
2                     Berlin
3                Brandenburg
4                     Bremen
5                    Hamburg
6                     Hessen
7                    Sachsen
8     Mecklenburg_Vorpommern
9        Nordrhein_Westfalen
10           Rheinland_Pfalz
11                  Saarland
12                   Sachsen
13            Sachsen_Anhalt
14        Schleswig_Holstein
15                 Thüringen
Name: state, dtype: object

In [247]:
state.reset_index(inplace = True)
state.head()

,index,state,area_km2,population_2019,population_per_km2,gdp_per_capita_2018,hdi
0,0,Baden_Württemberg,35752,11100394,310,47290,0.953
1,1,Bayern,70552,13124737,185,48323,0.947
2,2,Berlin,892,3669491,4086,41967,0.950
3,3,Brandenburg,29479,2521893,85,29541,0.914
4,4,Bremen,419,681202,1630,49215,0.951


In [248]:
new_state = pd.merge(state, state_supply, on = 'state', how = 'inner')
new_state

,index,state,area_km2,population_2019,population_per_km2,gdp_per_capita_2018,hdi,total_state_listings,total_state_sqm
0,0,Baden_Württemberg,35752,11100394,310,47290,0.953,15961,1342806.70
1,1,Bayern,70552,13124737,185,48323,0.947,21490,1718920.03
2,2,Berlin,892,3669491,4086,41967,0.950,10271,842022.42
3,3,Brandenburg,29479,2521893,85,29541,0.914,6875,474904.52
4,4,Bremen,419,681202,1630,49215,0.951,2940,213666.12
5,5,Hamburg,755,1847253,2439,66879,0.975,3730,312253.87
6,6,Hessen,21115,6288080,297,46923,0.949,17682,1466268.24
7,7,Sachsen,47609,7993448,168,38423,0.922,57673,3837202.43
8,12,Sachsen,18416,4077937,221,31453,0.930,57673,3837202.43
9,8,Mecklenburg_Vorpommern,23180,1609675,69,28940,0.910,6580,416557.76


In [249]:
new_state.drop(columns = ['index'], inplace = True)
new_state.head()

,state,area_km2,population_2019,population_per_km2,gdp_per_capita_2018,hdi,total_state_listings,total_state_sqm
0,Baden_Württemberg,35752,11100394,310,47290,0.953,15961,1342806.70
1,Bayern,70552,13124737,185,48323,0.947,21490,1718920.03
2,Berlin,892,3669491,4086,41967,0.950,10271,842022.42
3,Brandenburg,29479,2521893,85,29541,0.914,6875,474904.52
4,Bremen,419,681202,1630,49215,0.951,2940,213666.12


In [250]:
new_state['listings_per_100kcapita'] = (new_state.total_state_listings / new_state.population_2019) * 100
new_state['listings_per_10ksqm'] = (new_state.total_state_sqm / new_state.population_2019) * 10
new_state

,state,area_km2,population_2019,population_per_km2,gdp_per_capita_2018,hdi,total_state_listings,total_state_sqm,listings_per_100kcapita,listings_per_10ksqm
0,Baden_Württemberg,35752,11100394,310,47290,0.953,15961,1342806.70,0.143788,1.209693
1,Bayern,70552,13124737,185,48323,0.947,21490,1718920.03,0.163737,1.309680
2,Berlin,892,3669491,4086,41967,0.950,10271,842022.42,0.279903,2.294657
3,Brandenburg,29479,2521893,85,29541,0.914,6875,474904.52,0.272613,1.883127
4,Bremen,419,681202,1630,49215,0.951,2940,213666.12,0.431590,3.136604
5,Hamburg,755,1847253,2439,66879,0.975,3730,312253.87,0.201921,1.690369
6,Hessen,21115,6288080,297,46923,0.949,17682,1466268.24,0.281199,2.331822
7,Sachsen,47609,7993448,168,38423,0.922,57673,3837202.43,0.721503,4.800435
8,Sachsen,18416,4077937,221,31453,0.930,57673,3837202.43,1.414269,9.409666
9,Mecklenburg_Vorpommern,23180,1609675,69,28940,0.910,6580,416557.76,0.408778,2.587838


In [251]:
new_state.drop(columns = ['population_2019', 'area_km2', 'total_state_listings', 'total_state_sqm'], inplace = True)
new_state

,state,population_per_km2,gdp_per_capita_2018,hdi,listings_per_100kcapita,listings_per_10ksqm
0,Baden_Württemberg,310,47290,0.953,0.143788,1.209693
1,Bayern,185,48323,0.947,0.163737,1.309680
2,Berlin,4086,41967,0.950,0.279903,2.294657
3,Brandenburg,85,29541,0.914,0.272613,1.883127
4,Bremen,1630,49215,0.951,0.431590,3.136604
5,Hamburg,2439,66879,0.975,0.201921,1.690369
6,Hessen,297,46923,0.949,0.281199,2.331822
7,Sachsen,168,38423,0.922,0.721503,4.800435
8,Sachsen,221,31453,0.930,1.414269,9.409666
9,Mecklenburg_Vorpommern,69,28940,0.910,0.408778,2.587838


In [252]:
rentals.head()

,state,serviceCharge,heatingType,telekomTvOffer,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,yearConstructed,...,typeOfFlat,zip_code,noRooms,thermalChar,numberOfFloors,garden,district,town_municipality,date,baseRent_per_sqm
0,Nordrhein_Westfalen,245.00,central_heating,ONE_YEAR_FREE,False,False,6,4.62,10.0,1965-01-01,...,ground_floor,44269,4.0,181.4,3.0,True,Dortmund,Schüren,2019-05-01,6.918605
1,Rheinland_Pfalz,134.00,self_contained_central_heating,ONE_YEAR_FREE,False,True,8,3.47,10.0,1871-01-01,...,ground_floor,67459,3.0,NaN,NaN,False,Rhein_Pfalz,Böhl_Iggelheim,2019-05-01,8.988764
2,Sachsen,255.00,floor_heating,ONE_YEAR_FREE,True,True,8,2.72,2.4,2019-01-01,...,apartment,1097,3.0,NaN,4.0,False,Dresden,Äußere_Neustadt_Antonstadt,2019-10-01,11.515513
3,Sachsen,58.15,district_heating,ONE_YEAR_FREE,False,True,9,1.53,40.0,1964-01-01,...,other,9599,3.0,86.0,NaN,False,Mittelsachsen,Freiberg,2019-05-01,5.898538
4,Bremen,138.00,self_contained_central_heating,NaN,False,True,19,2.46,NaN,1950-01-01,...,apartment,28213,3.0,188.9,NaN,False,Bremen,Neu_Schwachhausen,2020-02-01,9.003178


In [230]:
rentals.shape

(266220, 30)

In [254]:
rentals.state

array(['Nordrhein_Westfalen', 'Rheinland_Pfalz', 'Sachsen', ..., 'Hessen',
       'Nordrhein_Westfalen', 'Hessen'], dtype=object)

In [259]:
a = set(rentals.state.values)

In [262]:
b = set(new_state.state.values)

In [264]:
a.difference(b)

{'Niedersachsen'}

In [231]:
df = pd.merge(rentals,new_state, on = 'state', how = 'left')
df.shape

(323893, 35)

In [186]:
df.tail()

,state,serviceCharge,heatingType,telekomTvOffer,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,yearConstructed,...,garden,district,town_municipality,date,baseRent_per_sqm,population_per_km2,gdp_per_capita_2018,hdi,listings_per_100kcapita,listings_per_10ksqm
323888,Bayern,90.0,heat_pump,ONE_YEAR_FREE,False,True,0,2.74,10.0,2016-01-01,...,False,Weilheim_Schongau,Eberfing,2020-02-01,9.111111,185.0,48323.0,0.947,0.163737,1.309680
323889,Hessen,220.0,gas_heating,NaN,False,True,12,6.49,NaN,1983-01-01,...,False,Bergstraße,Viernheim,2019-05-01,8.086957,297.0,46923.0,0.949,0.281199,2.331822
323890,Hessen,220.0,central_heating,ONE_YEAR_FREE,False,True,21,2.90,40.0,1965-01-01,...,True,Limburg_Weilburg,Limburg_an_der_Lahn,2020-02-01,6.842105,297.0,46923.0,0.949,0.281199,2.331822
323891,Nordrhein_Westfalen,175.0,heat_pump,NaN,True,True,16,4.39,NaN,2019-01-01,...,False,Köln,Dellbrück,2019-05-01,12.000000,526.0,39678.0,0.936,0.346123,2.573886
323892,Hessen,315.0,central_heating,NaN,False,True,9,5.00,NaN,1972-01-01,...,False,Frankfurt_am_Main,Sossenheim,2020-02-01,12.142857,297.0,46923.0,0.949,0.281199,2.331822


In [142]:
rentals.shape

(307489, 35)

In [ ]:
_= msno.bar(new_rentals, sort = 'ascending')

In [ ]:
missing = new_rentals.loc[:,new_rentals.isnull().any()]
missing.head()

In [ ]:
missing.columns

In [ ]:
missing.shape

In [ ]:
new_rentals.shape